This is a script for converting the variables (visibility observations and forecasts, and air temperature forecasts) in the EUPPBench dataset from netCDF format into a dataframe of records by flattening. This data preparation is for the validation (2017) and test (2018) sets.

In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import datetime

In [ ]:
data_path = "./"
export_path = "./"

In [2]:
# Loading the data in netCDF format
vis_forecast_obs = xr.open_dataarray(data_path + "vis_forecast_observations.nc")
vis_forecast = xr.open_dataarray(data_path + "vis_forecasts.nc")
t_forecast = xr.open_dataarray(data_path + "t_forecasts.nc")

/Users/clifflau/opt/anaconda3/lib/python3.8/site-packages/gribapi/__init__.py:23: UserWarning: ecCodes 2.31.0 or higher is recommended. You are running version 2.30.0
  warnings.warn(


In [3]:
# Data structure
# vis_obs['forecast_period', spot_index', 'forecast_reference_time']
# vis_reforecast['forecast_period', 'realization', 'spot_index', 'forecast_reference_time']
vis_forecast

<xarray.DataArray 'visibility_in_air' (forecast_period: 21, realization: 51,
                                       spot_index: 122,
                                       forecast_reference_time: 730)>
[95383260 values with dtype=float32]
Coordinates: (12/16)
  * forecast_period          (forecast_period) timedelta64[ns] 0 days 00:00:0...
  * realization              (realization) int64 0 1 2 3 4 5 ... 46 47 48 49 50
  * spot_index               (spot_index) int64 0 1 2 3 4 ... 118 119 120 121
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 2017-01...
    altitude                 (spot_index) float64 ...
    day_of_year              (forecast_reference_time) int64 ...
    ...                       ...
    model_land_usage         (spot_index) int8 ...
    model_latitude           (spot_index) float64 ...
    model_longitude          (spot_index) float64 ...
    model_orography          (spot_index) float64 ...
    surface                  float64 ...
    wmo_id                   (spot_index) int64 ...
Attributes:
    standard_name:  visibility_in_air
    units:          m

In [4]:
# Create arrays for latitude, longitudes, base times, lead times and orecasted/observed values
lat = np.array(vis_forecast_obs['latitude'])
lon = np.array(vis_forecast_obs['longitude'])
alt = np.array(vis_forecast_obs['altitude'])
base= np.array(vis_forecast_obs['forecast_reference_time'])
lead = np.array(vis_forecast_obs['forecast_period'])
lead_hr = np.array(lead*10**-9/3600, dtype='int')
vis_forecast_obs_array = np.array(vis_forecast_obs)
vis_forecast_array = np.array(vis_forecast[:, :, 0:118, :])
t_forecast_array = np.array(t_forecast[:, :, 0:118, :])

In [5]:
# Compute ensemble mean
import time

st = time.time()

vis_forecast_array_ensemble = vis_forecast_array
t_forecast_array_ensemble = t_forecast_array

for l in range(len(lead)):
    for s in range(len(lon)):
        for b in range(len(base)):
            vis_forecast_array_ensemble[l,:,s,b] = np.mean(vis_forecast_array_ensemble[l,:,s,b])
            t_forecast_array_ensemble[l,:,s,b] = np.mean(t_forecast_array_ensemble[l,:,s,b])

vis_forecast_array_ensemble = vis_forecast_array_ensemble[:, 0, 0:118, :]
t_forecast_array_ensemble = t_forecast_array_ensemble[:, 0, 0:118, :]

et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

Execution time: 30.80338215827942 seconds


In [6]:
# Check dimensions
vis_forecast_obs_array.shape, t_forecast_array_ensemble.shape, vis_forecast_array_ensemble.shape

((21, 118, 730), (21, 118, 730), (21, 118, 730))

In [7]:
# Create arrays for writting a dataframe of records
base_df = np.tile(base, len(lat)*len(lead_hr))
lon_df = np.tile(np.repeat(lon, len(base)), len(lead_hr))
lat_df = np.tile(np.repeat(lat, len(base)), len(lead_hr))
alt_df = np.tile(np.repeat(alt, len(base)), len(lead_hr))
lead_df = np.repeat(lead, len(base)*len(lat))
lead_hr_df = np.repeat(lead_hr, len(base)*len(lat))
forecast_time_df = base_df + lead_df
forecast_time_pd = pd.DatetimeIndex(forecast_time_df)
time_in_day = [i.hour for i in forecast_time_pd]
vis_forecast_obs_df = vis_forecast_obs_array.flatten()
vis_forecast_df = vis_forecast_array_ensemble.flatten()
t_forecast_df = t_forecast_array_ensemble.flatten()

In [8]:
# Create the dataframe
df = pd.DataFrame({'base': base_df,
                   'lead': lead_df,
                   'lead_hr': lead_hr_df,
                   'forecast_time': forecast_time_df,
                   'time_in_day': time_in_day,
                   'station_lat': lat_df,
                   'station_lon': lon_df,
                   'station_alt': alt_df,
                   't_forecast': t_forecast_df,
                   'vis_forecast': vis_forecast_df,
                   'vis_obs': vis_forecast_obs_df})

df

,base,lead,lead_hr,forecast_time,time_in_day,station_lat,station_lon,station_alt,t_forecast,vis_forecast,vis_obs
0,2017-01-01,0 days,0,2017-01-01,0,52.928000,4.781000,1.2,272.337158,21243.568359,4500.0
1,2017-01-02,0 days,0,2017-01-02,0,52.928000,4.781000,1.2,268.479401,22479.310547,20000.0
2,2017-01-03,0 days,0,2017-01-03,0,52.928000,4.781000,1.2,268.283478,20113.712891,25000.0
3,2017-01-04,0 days,0,2017-01-04,0,52.928000,4.781000,1.2,271.488556,16410.386719,11000.0
4,2017-01-05,0 days,0,2017-01-05,0,52.928000,4.781000,1.2,267.006775,20657.412109,20000.0
...,...,...,...,...,...,...,...,...,...,...,...
1808935,2018-12-27,5 days,120,2019-01-01,0,45.786833,3.149333,331.0,279.429871,17238.462891,6715.0
1808936,2018-12-28,5 days,120,2019-01-02,0,45.786833,3.149333,331.0,278.777802,16497.964844,15318.0
1808937,2018-12-29,5 days,120,2019-01-03,0,45.786833,3.149333,331.0,268.174530,44745.261719,33728.0
1808938,2018-12-30,5 days,120,2019-01-04,0,45.786833,3.149333,331.0,267.624115,35814.320312,38492.0


In [9]:
# Dropping extreme observations
df_drop = df.drop(df[(df.vis_obs>100000)|(df.vis_forecast>100000)].index)

In [10]:
# Validation period: 2017
# Testing period: 2018
df_valid = df_drop[df_drop['base']<=np.datetime64('2017-12-31')]
df_test = df_drop[df_drop['base']>np.datetime64('2017-12-31')]

In [11]:
df_valid

,base,lead,lead_hr,forecast_time,time_in_day,station_lat,station_lon,station_alt,t_forecast,vis_forecast,vis_obs
0,2017-01-01,0 days,0,2017-01-01,0,52.928000,4.781000,1.2,272.337158,21243.568359,4500.0
1,2017-01-02,0 days,0,2017-01-02,0,52.928000,4.781000,1.2,268.479401,22479.310547,20000.0
2,2017-01-03,0 days,0,2017-01-03,0,52.928000,4.781000,1.2,268.283478,20113.712891,25000.0
3,2017-01-04,0 days,0,2017-01-04,0,52.928000,4.781000,1.2,271.488556,16410.386719,11000.0
4,2017-01-05,0 days,0,2017-01-05,0,52.928000,4.781000,1.2,267.006775,20657.412109,20000.0
...,...,...,...,...,...,...,...,...,...,...,...
1808570,2017-12-27,5 days,120,2018-01-01,0,45.786833,3.149333,331.0,276.954529,39495.140625,60000.0
1808571,2017-12-28,5 days,120,2018-01-02,0,45.786833,3.149333,331.0,271.899323,23193.568359,59895.0
1808572,2017-12-29,5 days,120,2018-01-03,0,45.786833,3.149333,331.0,277.643158,24519.666016,60000.0
1808573,2017-12-30,5 days,120,2018-01-04,0,45.786833,3.149333,331.0,272.769440,33135.472656,59260.0


In [12]:
df_test

,base,lead,lead_hr,forecast_time,time_in_day,station_lat,station_lon,station_alt,t_forecast,vis_forecast,vis_obs
365,2018-01-01,0 days,0,2018-01-01,0,52.928000,4.781000,1.2,273.007294,42355.617188,8000.0
366,2018-01-02,0 days,0,2018-01-02,0,52.928000,4.781000,1.2,270.644257,34926.656250,20000.0
367,2018-01-03,0 days,0,2018-01-03,0,52.928000,4.781000,1.2,274.187927,1567.601196,5000.0
368,2018-01-04,0 days,0,2018-01-04,0,52.928000,4.781000,1.2,272.069672,38424.871094,10000.0
369,2018-01-05,0 days,0,2018-01-05,0,52.928000,4.781000,1.2,271.073914,45295.449219,17000.0
...,...,...,...,...,...,...,...,...,...,...,...
1808935,2018-12-27,5 days,120,2019-01-01,0,45.786833,3.149333,331.0,279.429871,17238.462891,6715.0
1808936,2018-12-28,5 days,120,2019-01-02,0,45.786833,3.149333,331.0,278.777802,16497.964844,15318.0
1808937,2018-12-29,5 days,120,2019-01-03,0,45.786833,3.149333,331.0,268.174530,44745.261719,33728.0
1808938,2018-12-30,5 days,120,2019-01-04,0,45.786833,3.149333,331.0,267.624115,35814.320312,38492.0


In [13]:
# Save into a csv files
df_valid.to_csv(export_path + "df_valid.csv")
df_test.to_csv(export_path + "df_test.csv")

In [15]:
# Check the records in the dataframe are matched correctly
i = 127182
print(df_test.iloc[i])
l = np.asarray(lead==df_test.iloc[i]['lead']).nonzero()[0]
s = np.asarray(lat==df_test.iloc[i]['station_lat']).nonzero()[0]
b = np.asarray(base==df_test.iloc[i]['base']).nonzero()[0]
print(int(lead[l]*10**-9/3600),
      float(lat[s]),
      base[b],
      int(alt[s]),
      float(np.mean(t_forecast[l,:,s,b])),
      float(np.mean(vis_forecast[l,:,s,b])),
      int(vis_forecast_obs[l,s,b]))

base             2018-10-22 00:00:00
lead                 0 days 18:00:00
lead_hr                           18
forecast_time    2018-10-22 18:00:00
time_in_day                       18
station_lat                49.209667
station_lon                 4.155333
station_alt                     95.0
t_forecast                279.748383
vis_forecast            49797.820312
vis_obs                      20000.0
Name: 340109, dtype: object
18 49.209667 ['2018-10-22T00:00:00.000000000'] 95 279.7483825683594 49797.8203125 20000
